In [2]:
# Importando bibliotecas necessárias
from IPython.display import clear_output
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network._multilayer_perceptron import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree._classes import DecisionTreeClassifier
from pactools.grid_search import GridSearchCVProgressBar
import tqdm
import multiprocessing
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [4]:
n_jobs = multiprocessing.cpu_count() - 1

In [4]:
periodo = 'pre-pandemia'
df = pd.read_csv(f'datasets/twitterbase_{periodo}.csv', sep=';')
cv = KFold(n_splits=10, shuffle=True, random_state=42)
zscore = MinMaxScaler()
x = df.drop('classe', axis=1).copy().abs()
y = df['classe'].copy()
#x = zscore.fit_transform(x)

k = 37
selector = SelectKBest(chi2, k=k)
X_new = selector.fit_transform(x, y)

# Obtenha os índices dos melhores atributos selecionados
indices_melhores_atributos = selector.get_support(indices=True)

# Obtenha os nomes dos melhores atributos
x.columns[indices_melhores_atributos]

Index(['volumeTweets_media', 'volumeTweets_variancia',
       'volumeTweets_mediaMovelPonterada', 'volumeTweets_entropia',
       'indiceInsonia_mediaMovelPonterada', 'indiceInsonia_entropia',
       'pronome1Pessoa_media', 'pronome1Pessoa_variancia',
       'pronome1Pessoa_mediaMovelPonterada', 'pronome1Pessoa_entropia',
       'pronome2Pessoa_mediaMovelPonterada', 'pronome3Pessoa_variancia',
       'pronome3Pessoa_mediaMovelPonterada', 'pronome3Pessoa_entropia',
       'valencia_variancia', 'valencia_entropia', 'ativacao_variancia',
       'ativacao_entropia', 'termosDepressivos_mediaMovelPonterada',
       'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada',
       'grafoSocial_entropia', 'caracteresOrientais_variancia',
       'emojis_mediaMovelPonterada', 'curtidas_media', 'curtidas_variancia',
       'curtidas_mediaMovelPonterada', 'hashtags_variancia',
       'hashtags_mediaMovelPonterada', 'retweets_media', 'retweets_variancia',
       'retweets_mediaMovelPonterada', 'me

In [5]:
atributos = x.columns[indices_melhores_atributos].tolist() + ['classe']
print(atributos)


['volumeTweets_media', 'volumeTweets_variancia', 'volumeTweets_mediaMovelPonterada', 'volumeTweets_entropia', 'indiceInsonia_mediaMovelPonterada', 'indiceInsonia_entropia', 'pronome1Pessoa_media', 'pronome1Pessoa_variancia', 'pronome1Pessoa_mediaMovelPonterada', 'pronome1Pessoa_entropia', 'pronome2Pessoa_mediaMovelPonterada', 'pronome3Pessoa_variancia', 'pronome3Pessoa_mediaMovelPonterada', 'pronome3Pessoa_entropia', 'valencia_variancia', 'valencia_entropia', 'ativacao_variancia', 'ativacao_entropia', 'termosDepressivos_mediaMovelPonterada', 'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada', 'grafoSocial_entropia', 'caracteresOrientais_variancia', 'emojis_mediaMovelPonterada', 'curtidas_media', 'curtidas_variancia', 'curtidas_mediaMovelPonterada', 'hashtags_variancia', 'hashtags_mediaMovelPonterada', 'retweets_media', 'retweets_variancia', 'retweets_mediaMovelPonterada', 'mencoes_media', 'mencoes_variancia', 'mencoes_mediaMovelPonterada', 'polaridade_entropia', 'subjetividade_

In [6]:
atributos = ['volumeTweets_media', 'volumeTweets_variancia', 'volumeTweets_mediaMovelPonterada', 'volumeTweets_entropia', 
 'indiceInsonia_mediaMovelPonterada', 'indiceInsonia_entropia', 
 'pronome1Pessoa_media', 'pronome1Pessoa_variancia', 'pronome1Pessoa_mediaMovelPonterada', 'pronome1Pessoa_entropia', 
 'pronome2Pessoa_mediaMovelPonterada', 
 'pronome3Pessoa_variancia', 'pronome3Pessoa_mediaMovelPonterada', 'pronome3Pessoa_entropia', 
 'valencia_variancia', 'valencia_entropia', 
 'ativacao_variancia', 'ativacao_entropia', 
 'termosDepressivos_mediaMovelPonterada', 
 'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada', 'grafoSocial_entropia',
 'medicamentosAntiDepressivo_variancia',
 'caracteresOrientais_variancia', 
 'emojis_mediaMovelPonterada', 
 'curtidas_media', 'curtidas_variancia', 'curtidas_mediaMovelPonterada',
 'midia_variancia',
 'links_entropia',
 'hashtags_variancia', 'hashtags_mediaMovelPonterada', 
                   'retweets_media', 'retweets_variancia', 'retweets_mediaMovelPonterada', 
 'mencoes_media', 'mencoes_variancia', 'mencoes_mediaMovelPonterada', 
 'polaridade_entropia', 
 'subjetividade_entropia', 'classe']

In [7]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)
zscore = StandardScaler()

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

df = pd.read_csv(f'datasets/twitterbase_{periodo}.csv', sep=';', usecols=atributos)

label_encoder = LabelEncoder()
df['classe'] = label_encoder.fit_transform(df['classe']).astype('float64')

x = df.drop('classe', axis=1).copy()
y = df['classe'].copy()
x = zscore.fit_transform(x)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=validation_ratio, random_state=1)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=1)



In [8]:
# Definindo o classificador (DecisionTreeClassifier)
dt_classifier = DecisionTreeClassifier(random_state=42, max_depth=10)

# Definindo os hiperparâmetros que queremos otimizar
param_grid = {
    'criterion': ['gini', 'entropy']
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=cv, scoring='f1', verbose=10)

# Realizando a busca pelos melhores hiperparâmetros
grid_search.fit(x_train, y_train)

clear_output()

# Obtendo os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores Hiperparâmetros:", best_params)

# Obtendo o modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_

# Fazendo previsões no conjunto de validacao
y_pred = best_model.predict(x_val)

# Calculando o f1-score do modelo no conjunto de validacao
f1score = f1_score(y_val, y_pred).round(2)
print("F1-score do Modelo:", f1score)

Melhores Hiperparâmetros: {'criterion': 'gini'}
F1-score do Modelo: 0.79


In [9]:
# Definindo o classificador (LinearSVC)
svc_classifier = LinearSVC(random_state=42, verbose=10, max_iter=100)

# Definindo os hiperparâmetros que queremos otimizar
param_grid = {
    'C': [0.1, 1.0, 10.0],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=svc_classifier, param_grid=param_grid, cv=cv, scoring='f1', verbose=10)

# Realizando a busca pelos melhores hiperparâmetros
grid_search.fit(x_train, y_train)

clear_output()

# Obtendo os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores Hiperparâmetros:", best_params)

# Obtendo o modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_

# Fazendo previsões no conjunto de validacao
y_pred = best_model.predict(x_val)

# Calculando o f1-score do modelo no conjunto de validacao
f1score = f1_score(y_val, y_pred).round(2)
print("F1-score do Modelo:", f1score)

Melhores Hiperparâmetros: {'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l2'}
F1-score do Modelo: 0.73


In [8]:
# Definindo o classificador (MLPClassifier)
mlp_classifier = MLPClassifier(random_state=42, max_iter=100, n_iter_no_change=20, batch_size=128, early_stopping=True, solver='adam', shuffle=True)

# Definindo os hiperparâmetros que queremos otimizar
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (10,30,10)],
    'activation': ['relu', 'tanh'],
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=mlp_classifier, param_grid=param_grid, cv=cv, scoring='f1', verbose=10)

# Realizando a busca pelos melhores hiperparâmetros
grid_search.fit(x_train, y_train)

clear_output()

# Obtendo os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores Hiperparâmetros:", best_params)

# Obtendo o modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_

# Fazendo previsões no conjunto de validacao
y_pred = best_model.predict(x_val)

# Calculando o f1-score do modelo no conjunto de validacao
f1score = f1_score(y_val, y_pred).round(2)
print("F1-score do Modelo:", f1score)

Melhores Hiperparâmetros: {'activation': 'relu', 'hidden_layer_sizes': (50, 50)}
F1-score do Modelo: 0.81


In [11]:
# Definindo o classificador (RandomForestClassifier)
rf_classifier = RandomForestClassifier(random_state=42, verbose=10, n_jobs=n_jobs, max_depth=10)

# Definindo os hiperparâmetros que queremos otimizar
param_grid = {
    'n_estimators': [10, 50, 100],
    'criterion': ['gini', 'entropy']
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=cv, scoring='f1', verbose=10)

# Realizando a busca pelos melhores hiperparâmetros
grid_search.fit(x_train, y_train)

clear_output()

# Obtendo os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores Hiperparâmetros:", best_params)

# Obtendo o modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_

# Fazendo previsões no conjunto de validacao
y_pred = best_model.predict(x_val)

# Calculando o f1-score do modelo no conjunto de validacao
f1score = f1_score(y_val, y_pred).round(2)
print("F1-score do Modelo:", f1score)

Melhores Hiperparâmetros: {'criterion': 'gini', 'n_estimators': 50}
F1-score do Modelo: 0.8


[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  11 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  43 out of  50 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  50 out of  50 | elapsed:    0.0s finished


In [5]:
periodo = 'pandemia'
df = pd.read_csv(f'datasets/twitterbase_{periodo}.csv', sep=';')
cv = KFold(n_splits=10, shuffle=True, random_state=42)
zscore = MinMaxScaler()
x = df.drop('classe', axis=1).copy().abs()
y = df['classe'].copy()
#x = zscore.fit_transform(x)

k = 37
selector = SelectKBest(chi2, k=k)
X_new = selector.fit_transform(x, y)

# Obtenha os índices dos melhores atributos selecionados
indices_melhores_atributos = selector.get_support(indices=True)

# Obtenha os nomes dos melhores atributos
x.columns[indices_melhores_atributos]

Index(['volumeTweets_media', 'volumeTweets_variancia',
       'volumeTweets_mediaMovelPonterada', 'volumeTweets_entropia',
       'indiceInsonia_variancia', 'indiceInsonia_mediaMovelPonterada',
       'indiceInsonia_entropia', 'pronome1Pessoa_media',
       'pronome1Pessoa_variancia', 'pronome1Pessoa_mediaMovelPonterada',
       'pronome1Pessoa_entropia', 'pronome2Pessoa_mediaMovelPonterada',
       'pronome3Pessoa_media', 'pronome3Pessoa_variancia',
       'pronome3Pessoa_mediaMovelPonterada', 'pronome3Pessoa_entropia',
       'valencia_mediaMovelPonterada', 'valencia_entropia',
       'ativacao_mediaMovelPonterada', 'ativacao_entropia',
       'termosDepressivos_variancia', 'grafoSocial_variancia',
       'grafoSocial_mediaMovelPonterada', 'grafoSocial_entropia',
       'caracteresOrientais_variancia', 'emojis_variancia', 'midia_variancia',
       'midia_mediaMovelPonterada', 'curtidas_media', 'curtidas_variancia',
       'curtidas_mediaMovelPonterada', 'hashtags_variancia',
       '

In [6]:
atributos = x.columns[indices_melhores_atributos].tolist() + ['classe']
print(atributos)

['volumeTweets_media', 'volumeTweets_variancia', 'volumeTweets_mediaMovelPonterada', 'volumeTweets_entropia', 'indiceInsonia_variancia', 'indiceInsonia_mediaMovelPonterada', 'indiceInsonia_entropia', 'pronome1Pessoa_media', 'pronome1Pessoa_variancia', 'pronome1Pessoa_mediaMovelPonterada', 'pronome1Pessoa_entropia', 'pronome2Pessoa_mediaMovelPonterada', 'pronome3Pessoa_media', 'pronome3Pessoa_variancia', 'pronome3Pessoa_mediaMovelPonterada', 'pronome3Pessoa_entropia', 'valencia_mediaMovelPonterada', 'valencia_entropia', 'ativacao_mediaMovelPonterada', 'ativacao_entropia', 'termosDepressivos_variancia', 'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada', 'grafoSocial_entropia', 'caracteresOrientais_variancia', 'emojis_variancia', 'midia_variancia', 'midia_mediaMovelPonterada', 'curtidas_media', 'curtidas_variancia', 'curtidas_mediaMovelPonterada', 'hashtags_variancia', 'hashtags_mediaMovelPonterada', 'retweets_variancia', 'mencoes_variancia', 'mencoes_mediaMovelPonterada', 'polar

In [7]:
atributos = ['volumeTweets_media', 'volumeTweets_variancia', 'volumeTweets_mediaMovelPonterada', 'volumeTweets_entropia', 
'indiceInsonia_variancia', 'indiceInsonia_mediaMovelPonterada', 'indiceInsonia_entropia', 
'pronome1Pessoa_media', 'pronome1Pessoa_variancia', 'pronome1Pessoa_mediaMovelPonterada', 'pronome1Pessoa_entropia', 
'pronome2Pessoa_mediaMovelPonterada', 
'pronome3Pessoa_media', 'pronome3Pessoa_variancia', 'pronome3Pessoa_mediaMovelPonterada', 'pronome3Pessoa_entropia', 
'valencia_mediaMovelPonterada', 'valencia_entropia', 
'ativacao_mediaMovelPonterada', 'ativacao_entropia', 
'termosDepressivos_variancia', 
'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada', 'grafoSocial_entropia',
'medicamentosAntiDepressivo_mediaMovelPonterada',
'caracteresOrientais_variancia', 
'emojis_variancia', 
'midia_variancia', 'midia_mediaMovelPonterada', 
'curtidas_media', 'curtidas_variancia', 'curtidas_mediaMovelPonterada',
'links_mediaMovelPonterada',
'hashtags_variancia', 'hashtags_mediaMovelPonterada', 
'retweets_variancia', 
'mencoes_variancia', 'mencoes_mediaMovelPonterada', 
'polaridade_entropia', 
'subjetividade_entropia', 'classe']

In [8]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)
zscore = StandardScaler()

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

df = pd.read_csv(f'datasets/twitterbase_{periodo}.csv', sep=';')

label_encoder = LabelEncoder()
df['classe'] = label_encoder.fit_transform(df['classe']).astype('float64')

x = df.drop('classe', axis=1).copy()
y = df['classe'].copy()
x = zscore.fit_transform(x)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=validation_ratio, random_state=1)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_ratio, random_state=1)



In [17]:
# Definindo o classificador (DecisionTreeClassifier)
dt_classifier = DecisionTreeClassifier(random_state=42, max_depth=10)

# Definindo os hiperparâmetros que queremos otimizar
param_grid = {
    'criterion': ['gini', 'entropy']
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=cv, scoring='f1', verbose=10)

# Realizando a busca pelos melhores hiperparâmetros
grid_search.fit(x_train, y_train)

clear_output()

# Obtendo os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores Hiperparâmetros:", best_params)

# Obtendo o modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_

# Fazendo previsões no conjunto de validacao
y_pred = best_model.predict(x_val)

# Calculando o f1-score do modelo no conjunto de validacao
f1score = f1_score(y_val, y_pred).round(2)
print("F1-score do Modelo:", f1score)

Melhores Hiperparâmetros: {'criterion': 'entropy'}
F1-score do Modelo: 0.77


In [18]:
# Definindo o classificador (LinearSVC)
svc_classifier = LinearSVC(random_state=42, verbose=10, max_iter=100)

# Definindo os hiperparâmetros que queremos otimizar
param_grid = {
    'C': [0.1, 1.0, 10.0],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=svc_classifier, param_grid=param_grid, cv=cv, scoring='f1', verbose=10)

# Realizando a busca pelos melhores hiperparâmetros
grid_search.fit(x_train, y_train)

clear_output()

# Obtendo os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores Hiperparâmetros:", best_params)

# Obtendo o modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_

# Fazendo previsões no conjunto de validacao
y_pred = best_model.predict(x_val)

# Calculando o f1-score do modelo no conjunto de validacao
f1score = f1_score(y_val, y_pred).round(2)
print("F1-score do Modelo:", f1score)

Melhores Hiperparâmetros: {'C': 1.0, 'loss': 'squared_hinge', 'penalty': 'l2'}
F1-score do Modelo: 0.74


In [9]:
# Definindo o classificador (MLPClassifier)
mlp_classifier = MLPClassifier(random_state=42, max_iter=100, n_iter_no_change=20, batch_size=128, early_stopping=True, solver='adam', shuffle=True)

# Definindo os hiperparâmetros que queremos otimizar
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (10,30,10)],
    'activation': ['relu', 'tanh'],
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=mlp_classifier, param_grid=param_grid, cv=cv, scoring='f1', verbose=10)

# Realizando a busca pelos melhores hiperparâmetros
grid_search.fit(x_train, y_train)

clear_output()

# Obtendo os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores Hiperparâmetros:", best_params)

# Obtendo o modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_

# Fazendo previsões no conjunto de validacao
y_pred = best_model.predict(x_val)

# Calculando o f1-score do modelo no conjunto de validacao
f1score = f1_score(y_val, y_pred).round(2)
print("F1-score do Modelo:", f1score)

Melhores Hiperparâmetros: {'activation': 'relu', 'hidden_layer_sizes': (50, 50)}
F1-score do Modelo: 0.8


In [19]:
# Definindo o classificador (RandomForestClassifier)
rf_classifier = RandomForestClassifier(random_state=42, verbose=10, n_jobs=n_jobs, max_depth=10)

# Definindo os hiperparâmetros que queremos otimizar
param_grid = {
    'n_estimators': [10, 50, 100],
    'criterion': ['gini', 'entropy']
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=cv, scoring='f1', verbose=10)

# Realizando a busca pelos melhores hiperparâmetros
grid_search.fit(x_train, y_train)

clear_output()

# Obtendo os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores Hiperparâmetros:", best_params)

# Obtendo o modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_

# Fazendo previsões no conjunto de validacao
y_pred = best_model.predict(x_val)

# Calculando o f1-score do modelo no conjunto de validacao
f1score = f1_score(y_val, y_pred).round(2)
print("F1-score do Modelo:", f1score)

Melhores Hiperparâmetros: {'criterion': 'gini', 'n_estimators': 50}
F1-score do Modelo: 0.8


[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  11 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done  43 out of  50 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  50 out of  50 | elapsed:    0.0s finished
